<a id="index"></a>
# A modular processing pipeline for spike sorting from axona raw data

In this notebook we will walk through a spike sorting processing pipeline with [SpikeInterface](https://github.com/SpikeInterface) for Axona raw (`.bin`) and axona unit (`.X`) data. There is another notebook for working with Intan data.

We will show how to

1. [Install `hussaini_lab_to_nwb`](#install_dependencies)
2. [load data with spikeextractors](#load_data)
3. [Preprocess the signals](#preprocess)
4. [Cache data](#cache)
5. [Spike sort](#spike_sorting)   
   * [Klusta](#klusta)
   * [mountainsort4](#mountainsort4)
   * [Ironclust](#ironclust)
   * [spyking circus](#spykingcircus)
   * [tridesclous](#tridesclous)
   
   
6. [Postprocess the signals](#postprocessing)
7. [Curate the spike sorting output](#spike_curation) using 
        1) quality metrics (automatic), 
        2) [Phy](https://github.com/cortex-lab/phy) (manual), and 
        3) consensus-based curation
8. [Export results to NWB](#export_nwb)
9. [Export results to TINT](#export_tint)


<a id='install_dependencies'></a>
# Install `hussaini_lab_to_nwb`
[back to index](#index)

You can install the package and its dependencies with `pip install hussaini_lab_to_nwb`. The dependencies are listed in `requirements.txt`. We recommend installing it in a clean virtual environment (e.g. with `conda` or `venv`) and install the requirements (`pip install -r requirements.txt`) before installing the package. 

Note that some spike sorters require matlab or additional dependencies, which need to be installed separately. For an overview of possible spike sorters and their dependencies see https://spikeinterface.readthedocs.io/en/latest/sortersinfo.html. The `requirements_sorters.txt` allows you to get "klusta", "spyking-circus", "tridesclous", and "mountainsort4" installed in one shot.

### Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# General data handling and visualization
import sys
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# SpikeInterface family
import spikeextractors as se
import spikewidgets as sw
import spiketoolkit as st
import spikesorters as ss
import spikecomparison as sc

print(sys.version, sys.platform, sys.executable)

In [ ]:
import neo
from neo.rawio import baserawio

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (16, 6)
plt.rcParams.update({'font.size':14})

<a id='load_data'></a>
# 2. Load data
[back to index](#index)

Set path and base filename (or `.set` filename) and create an `AxonaRecordingExtractor` (using `.bin` file) or `AxonaUnitRecordingExtractor` (using `.X` files) instance. It is assumed that the files are in the same directory.

In [ ]:
use_raw_data = False  # False for reading `.X` data, True for reading `.bin` data

In [ ]:
if use_raw_data:
    dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
    base_filename = '20201004_Raw'
else:
    dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint_no_bin'
    base_filename = 'axona_sample'

filename = os.path.join(dir_name, base_filename + '.set')
print(filename)

When reading `.X` data, `AxonaUnitRecordingExtractor` fills in the missing data between waveforms with Gaussian noise, which is required for spike sorting later on. The noise standard deviation can be set with the `noise_std` parameter.

In [ ]:
if use_raw_data:
    recording = se.AxonaRecordingExtractor(filename=filename)
else:
    recording = se.AxonaUnitRecordingExtractor(filename=filename, noise_std=15)

A `RecordingExtractor` object extracts information about channel ids, channel locations (if present), the sampling frequency of the recording, and the extracellular traces (when prompted). Below we illustrate a few useful methods that describe this particular recording.

In [ ]:
print('Type of recorindextractor:', type(recording))
print('Channel ids: {}'.format(recording.get_channel_ids()))
print('Num. channels = {}'.format(len(recording.get_channel_ids())))
print('Sampling frequency = {} Hz'.format(recording.get_sampling_frequency()))
print('Num. samples = {}'.format(recording.get_num_frames()))
print('Stdev. on third channel = {}'.format(np.std(recording.get_traces(
    start_frame=0, end_frame=48000, channel_ids=2))))

The `spikewidgets` module includes several convenient plotting functions that can be used to explore the data, for instance by visualizing snippets of data in the time domain:

In [ ]:
w_ts = sw.plot_timeseries(recording, trange=[0.4, 0.8])

Or in the frequency domain:

In [ ]:
w_sp = sw.plot_spectrum(recording, trange=[0, 5], channels=[1, 2, 3, 4, 5, 6, 7, 8])

It often makes sense to group electrodes together, for instance if they belong to the same tetrode. When reading Axona data channels are automatically grouped by tetrode membership by reading the information form the `.set` file. If desired, or if using a different recording system where groups are not automatically assigned, the `set_channel_groups()` method can be used. Correspondingly, we can view the `group` property using the `get_channel_groups` method. 

An alternative way to obtain channel groups is by loading a so called `.prb` file containing channel grouping and physical channel locations. We do not need it here, but it is well explained in this [spikeinterface tutorial](https://github.com/SpikeInterface/spiketutorials/blob/master/NWB_Developer_Breakout_Session_Sep2020/SpikeInterface_Tutorial.ipynb).

In [ ]:
print(f'Channel groups:\n{recording.get_channel_groups()}')

Some spike sorters (e.g. ironclust) require channel locations. So we will create a fake geometry keeping electrodes on single tetrodes close together. One could use a `.prb` file to accomplish this, but we will simply assign locations with the `set_channel_locations()` method. If the actual geometry is known, you can of course use that instead. 

In [ ]:
recording.set_channel_locations(locations=[[0,0],[0,1],[0,2],[0,3],
                                           [0,10],[0,11],[0,12],[0,13],
                                           [0,20],[0,21],[0,22],[0,23],
                                           [0,30],[0,31],[0,32],[0,33]],
                                channel_ids=recording.get_channel_ids())

We now have the additional channel property `location`.

In [ ]:
print('Channel ids:', recording.get_channel_ids())
print('Loaded properties', recording.get_shared_channel_property_names())

# 'group' and 'location' can be returned as lists:
print('Channel groups:', recording.get_channel_groups())
print('Channel locations:\n', recording.get_channel_locations())

And we can visualize the electrode locations with spikewidgets:

In [ ]:
fh = plt.figure(figsize=(1,6))
w_elec = sw.plot_electrode_geometry(recording, figure=fh)

### Properties (and features)

For now we have seen that the `RecordingExtractor` can have `group` and `location` *properies*. These are very special properties that can be very important for spike sorting. But anything related to a channel can be saved as a property.

Similarly, for `SortingExtractor` objects, anything related to a unit can be stored as a property. In addition, for `SortingExtractor` objects we can also store anything related to spikes as *features* (e.g. waveforms, as we'll see later).

We can check which properties are in the extractor as follows:

In [ ]:
print('Shared channel property names')
print(recording.get_shared_channel_property_names())

<a id='preprocess'></a>
# 3. Preprocessing
[back to index](#index)

Next, we will perform a few preprocessing steps using `spiketoolkit`.

We can filter the recordings, rereference the signals to remove noise, discard noisy channels, whiten the data, remove stimulation artifacts, and more (see [here](https://spiketoolkit.readthedocs.io/en/latest/preprocessing_example.html)).

For this notebook, we will filter the recordings, apply common median reference (CMR) and remove noisy channels (for illustration purposes). All preprocessing modules return new `RecordingExtractor` objects that apply the underlying preprocessing function. This allows users to access the preprocessed data in the same way as the raw data.

### Bandpass filter

In [ ]:
r_filt = st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000)

w = sw.plot_timeseries(r_filt, trange=[0.4, 0.8], color_groups=True)

### Common median reference (CMR)

It is often desirable to use a common median reference to remove noise common to all channels.

In [ ]:
r_cmr = st.preprocessing.common_reference(r_filt, reference='median')
w = sw.plot_timeseries(r_cmr, trange=[0.4, 0.8], color_groups=True)

### Remove bad channels

Some channels look more noisy than others. This could negatively influence the spike sorters, so let's remove some of the noisier channels (of course for actual analyses it might be better to use a more objective approach). We can do so as follows (alternatively we could create a `SubRecordingExtractor` and specify the channel IDs to keep):

In [ ]:
channel_ids = np.array(r_cmr.get_channel_ids())
r_good_channels = st.preprocessing.remove_bad_channels(r_cmr, bad_channel_ids=channel_ids[[0, 3, 5]])
print(f'Channel ids after removing bad channel: {r_good_channels.get_channel_ids()}')
print(f'Channel groups after removing bad channel: {r_good_channels.get_channel_groups()}')

In [ ]:
w = sw.plot_timeseries(r_good_channels, trange=[0.4, 0.8], color_groups=True)

We can also select subsets of data in the time domain with `SubRecordingExtractor`. For illustrative purposes and to speed up computation time of the spike sorters, we will select a 2 min sub-window. 


In [ ]:
fs = r_cmr.get_sampling_frequency()
r_sub = se.SubRecordingExtractor(r_good_channels, start_frame=0*fs, end_frame=120*fs)

<a id='cache'></a>
# 4. Cache data
[back to index](#index)

All operations in SpikeInterface are *lazy*, meaning that they are not performed if not needed. This is why the creation of our filter recording was almost instantaneous. However, to speed up further processing, we might want to perform those operations (eg. filters, CMR, etc.) once and **cache** the processed data to a file. This is particularly important if we are going to extract waveforms, templates, pca scores, or in general *post-process* the results.

Note that if the parameter `save_path` is not specified, the cache file will be saved in a temporary folder that will be deleted when the Python session is closed (or the object destroyed). One can get around this by moving the temporary file with `recording_cache.move_to()`, or simply specifying the `save_path` paramter when creating the cache. You can view the directory of the temporary folder with `recording_cache.get_tmp_folder()`.

In [ ]:
perma_cache_filename = os.path.join(dir_name, 'cached_data_preproc.dat') 
r_cache = se.CacheRecordingExtractor(r_sub, save_path = perma_cache_filename)
print('Temporary folder:\n{}'.format(r_cache.get_tmp_folder()))

Under the hood, this convenient function is retrieving all the traces (in chunks, to save up some memory), applying the preprocessing steps, and dumping them to a binary temporary file:

In [ ]:
print(f'Filename: {r_cache.filename}')

The cached recording has all the previously loaded information:

In [ ]:
print(f'Cached channels: {r_cache.get_channel_ids()}')
print(f'Cached channels ids: {r_cache.get_channel_ids()}')
print(f'Channel groups after caching: {r_cache.get_channel_groups()}')
print('Duration of recording in seconds: {}'.format(r_cache.get_num_frames() / r_cache.get_sampling_frequency()))

If we now closed the Python session, we would have a nice `.dat` file, but no information on how to open it! 
In order to save the state of an extractor, we can use the **dumping** mechanism.
Each extractor can be converted to a dictionary, which holds the path to the data file and all relevant information:

In [ ]:
r_cache.dump_to_dict()

We can now dump our extractor object, so it can be loaded in a future session. We can dump either to `.json` or to `.pkl`. Dumping to pickle also allows us to store properties (other than group and locations) and features (for `SortingExtractor` objects).

In [ ]:
r_cache.dump_to_pickle(os.path.join(dir_name, 'cached_data_preproc.pkl'))

In another session, we can pick up from where we left by loading the extractor from the pickle file:

In [ ]:
r_cache = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_data_preproc.pkl'))

In [ ]:
print(f'Cached channels ids: {r_cache.get_channel_ids()}')
print(f'Channel groups after caching: {r_cache.get_channel_groups()}')
print('Duration of recording in seconds: {}'.format(r_cache.get_num_frames() / r_cache.get_sampling_frequency()))

__Note__: the same caching/dumping mechanisms are available also for all `SortingExtractor` objects.

<a id='spike_sorting'></a>
# 5. Spike sorting
[back to index](#index)

We will now run spike sorting on the above recording (`r_cache`). We will show code for `klusta`, `ironclust`, `mountainsort4`, `spykingcircus` and `trideclous` for this demonstration, but there are many other spike sorting algorithms available, all of which can be accessed with the same `spikeinterface` API.

First, we need to install the spike sorters we want. There is a complete overview of spike sorters supported by spikeinterface [here](https://spikeinterface.readthedocs.io/en/latest/sortersinfo.html), including installation instructions. Note that not all spikesorters work on every setup. Some require Matlab. Some require CUDA. 

We can list all installed spike sorting algorithms with the following convenience function.

In [ ]:
ss.installed_sorters()

<a id='klusta'></a>
### Klusta
[back to index](#index)

We can retrieve the parameters associated with any sorter with the `get_default_params()` function from the `spikesorters` module, as well as corresponding parameter descriptions using `get_params_description()`.

In [ ]:
ss.get_default_params('klusta')

In [ ]:
ss.get_params_description('klusta')

In [ ]:
ss.run_sorter?

In [ ]:
ss.run_klusta?

Since we have tetrodes and we know that they are physically apart, we would like to sort them separately as follows, assigning the `group` property to the `grouping_property` parameter:

In [ ]:
sorting_KL = ss.run_klusta(
    r_cache,
    adjacency_radius=5,
    output_folder=os.path.join(dir_name, 'klusta_by_group'),
    grouping_property='group',
    parallel=True
)
print(f'Klusta found {len(sorting_KL.get_unit_ids())} units')

Below you can see that when the `grouping_property` is used a `MultiSortingExtractor` object is returned, containing several `SortingExtractor` objects, which were created separately for each group. 

In [ ]:
print(type(sorting_KL))
print(type(sorting_KL.sortings[0]))

In [ ]:
print(f'Klusta unit ids (with groups): {sorting_KL.get_unit_ids()}')

In [ ]:
print(f'Spike train of a unit: {sorting_KL.get_unit_spike_train(1)}')

We can also visually inspect the data using raster plots.

In [ ]:
w_rs = sw.plot_rasters(sorting_KL, trange=[0.4, 0.8])

We can plot the temporal data for comparison, though it can be quite difficult to visually make out spiking events in noisy data. We will see more rigorous ways for assessing spike sorting outputs later on.

In [ ]:
w = sw.plot_timeseries(r_cache, trange=[0.4, 0.8], color_groups=True)

### Loading spike sorting output from a spike sorting folder

If a spike sorter has been run, you can reload the output as a `SortingExtractor` using the corresponding `spikeextractors` class. Note that if sorting by group/property, single groups must be loaded separately:

In [ ]:
sorting_KL_grp = []
for i in range(0, 3):
    sorting_KL_grp.append(se.KlustaSortingExtractor(os.path.join(dir_name, 'klusta_by_group', str(i))))

In [ ]:
sorting_KL_grp
sorting_KL = se.MultiSortingExtractor(sorting_KL_grp)

In [ ]:
sorting_KL

In [ ]:
print(f'Klusta unit ids group 0: {sorting_KL_grp[0].get_unit_ids()}')

### Saving and loading spike sorting output

It is also possible to save spike sorting output in a format of a different spike sorter using the `write_sorting()` method of the sorting extractor of interest. For instance, below we are saving the output from `klusta` in `mountainsort` format. 

In [ ]:
sort_cache = se.CacheSortingExtractor(sorting_KL, save_path=os.path.join(dir_name, 'cached_sorting_KL.npz'))

sort_cache.dump_to_pickle(os.path.join(dir_name, 'cached_sorting_KL.pkl'))

Re-read data from file.

In [ ]:
sorting_KL = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_sorting_KL.pkl'))

print('Unit ids = {}'.format(sorting_KL.get_unit_ids()))
spike_train = sorting_KL.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(spike_train)))
spike_train1 = sorting_KL.get_unit_spike_train(unit_id=1, start_frame=0, end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(spike_train1)))

<a id='mountainsort4'></a>
### Mountainsort4
[back to index](#index)

Let's see if mountainsort is indeed installed.

In [ ]:
ss.installed_sorters()

In [ ]:
ss.get_default_params('mountainsort4')

In [ ]:
ss.get_params_description('mountainsort4')

In [ ]:
sorting_MS4 = ss.run_mountainsort4(
    r_cache,
    os.path.join(dir_name, 'mountainsort4_group'),
    grouping_property='group',
    adjacency_radius=5,
    parallel=False,  # on my local machine mountainsort fails when parallel=True, that is not generally the case
    verbose=True,
    filter=False
)
print(f'Mountainsort4 found {len(sorting_MS4.get_unit_ids())} units')

In [ ]:
print(f'Spike train of a unit: {sorting_MS4.get_unit_spike_train(1)}')

Write spike sorting output to disk.

In [ ]:
sort_cache = se.CacheSortingExtractor(sorting_MS4, save_path=os.path.join(dir_name, 'cached_sorting_MS4.npz'))

sort_cache.dump_to_pickle(os.path.join(dir_name, 'cached_sorting_MS4.pkl'))

Load spike sorting output from disk.

In [ ]:
sorting_MS4 = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_sorting_MS4.pkl'))

<a id='ironclust'></a>
### Ironclust
[back to index](#index)

If you haven't already, clone Ironclust github repository to a path of your choice. Here we stay in the same directory. 

In [ ]:
ss.get_default_params('ironclust')

In [ ]:
ss.get_params_description('ironclust')

Next, we set the ironclust path.

In [ ]:
ss.IronClustSorter.set_ironclust_path('./ironclust')  # You might need to substitute the appropriate path here
ss.IronClustSorter.ironclust_path

Note that we can also set a global environment variable called `IRONCLUST_PATH`. In that case we don't need to set the path in each session because the sorter class looks for this environment variable.

Now ironclust should be installed and we can run it:

In [ ]:
ss.installed_sorters()

In [ ]:
sorting_IC = ss.run_ironclust(
    r_cache, 
    os.path.join(dir_name, 'ironclust_group'), 
    grouping_property='group', 
    parallel=True, 
    verbose=True,
    filter=False,  # we already filtered
    common_ref_type=None  # we already applied median reference
)
print(f'IronClust found {len(sorting_IC.get_unit_ids())} units')

Cache sorting extractor.

In [ ]:
sort_cache = se.CacheSortingExtractor(sorting_IC, save_path=os.path.join(dir_name, 'cached_sorting_IC.npz'))

sort_cache.dump_to_pickle(os.path.join(dir_name, 'cached_sorting_IC.pkl'))

Re-load ironclust sorting extractor from file.

In [ ]:
sorting_IC = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_sorting_IC.pkl'))

<a id='spykingcircus'></a>
### SpykingCircus
[back to index](#index)

In [ ]:
ss.get_default_params('spykingcircus')

In [ ]:
ss.get_params_description('spykingcircus')

In [ ]:
ss.installed_sorters()

In [ ]:
sorting_SC = ss.run_spykingcircus(
    r_cache,
    os.path.join(dir_name, 'spykingcircus_group'),
    grouping_property='group',
    adjacency_radius=5,
    parallel=True,
    verbose=True,
    filter=False
)
print(f'Spyking Circus found {len(sorting_SC.get_unit_ids())} units')

Save spike sorting output.

In [ ]:
sort_cache = se.CacheSortingExtractor(sorting_SC, save_path=os.path.join(dir_name, 'cached_sorting_SC.npz'))

sort_cache.dump_to_pickle(os.path.join(dir_name, 'cached_sorting_SC.pkl'))

Read spyking circus spike sorting output. 

In [ ]:
sorting_SC = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_sorting_SC.pkl'))

print('Unit ids = {}'.format(sorting_SC.get_unit_ids()))
spike_train = sorting_SC.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(spike_train)))
spike_train1 = sorting_SC.get_unit_spike_train(unit_id=1, start_frame=0, end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(spike_train1)))

<a id='tridesclous'></a>
### Tridesclous
[back to index](#index)

If you haven't already, clone Ironclust github repository to a path of your choice. 

In [ ]:
ss.get_default_params('tridesclous')

In [ ]:
ss.get_params_description('tridesclous')

In [ ]:
# run spike sorting by group
sorting_TRI = ss.run_tridesclous(
    r_cache, 
    os.path.join(dir_name, 'tridescluous_group'), 
    grouping_property='group', 
    parallel=True, 
    verbose=True
)
print(f'Tridesclous found {len(sorting_TRI.get_unit_ids())} units')

Export spike sorting output from tridesclous.

In [ ]:
sort_cache = se.CacheSortingExtractor(sorting_TRI, save_path=os.path.join(dir_name, 'cached_sorting_TRI.npz'))

sort_cache.dump_to_pickle(os.path.join(dir_name, 'cached_sorting_TRI.pkl'))

Import spike sorting output from tridesclous.

In [ ]:
sorting_TRI = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_sorting_TRI.pkl'))

print('Unit ids = {}'.format(sorting_TRI.get_unit_ids()))
spike_train = sorting_TRI.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(spike_train)))
spike_train1 = sorting_TRI.get_unit_spike_train(unit_id=1, start_frame=0, end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(spike_train1)))

<a id='postprocessing'></a>
# 6. Postprocessing
[back to index](#index)

The `postprocessing` submodule of `spiketoolkit` allows us to extract information from the combination of the recording and sorting extractors. For example, we can extract waveforms, templates, maximum channels and pca scores. In addition, we can compute waveform features that could be used for further processing, e.g. classyfing excitatory-inhibitory neurons. 

Feel free to also browse the postprocessing [documentation](https://spikeinterface.readthedocs.io/en/latest/modules/toolkit/plot_2_postprocessing.ht) for more details.

Let's start by extracting waveforms.

In [ ]:
st.postprocessing.get_unit_waveforms?

In [ ]:
waveforms = st.postprocessing.get_unit_waveforms(r_cache, sorting_MS4, ms_before=0.2, ms_after=0.8, verbose=True)

In [ ]:
waveforms[1].shape

In [ ]:
avg_waveforms = st.postprocessing.get_unit_templates(r_cache, sorting_MS4, max_spikes_per_unit=200,
                                                     save_as_property=True, verbose=True)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,8))

ax[0].plot(waveforms[0][:, 0, :].T, color='k', lw=0.3)
ax[0].plot(waveforms[1][:, 0, :].T, color='r', lw=0.3)
ax[0].plot(waveforms[2][:, 0, :].T, color='b', lw=0.3)
ax[0].set_xlabel('Sample', fontsize=16)
ax[0].set_ylabel('Amplitude', fontsize=16)
ax[0].set_title('Individual waveforms for three sample units', fontsize=16)

ax[1].plot(avg_waveforms[0][0].T, color='k', lw=0.3)
ax[1].plot(avg_waveforms[1][0].T, color='r', lw=0.3)
ax[1].plot(avg_waveforms[2][0].T, color='b', lw=0.3)
ax[1].set_xlabel('Sample', fontsize=16)
ax[1].set_title('Average waveforms for three sample units', fontsize=16)

Similarly, we can get templates, maximum channels, and pca scores. 
When these are computed, they are automatically stored in the `SortingExtractor` object, so that they don't need to be recomputed. 

Each waveform is associated with a specific spike, so they are saved as spike *features*:


You may have noticed that 300 waveforms were extracted from the spike train of the first unit. However, it has more spikes:

In [ ]:
len(sorting_MS4.get_unit_spike_train(1))

This is because it can be convenient to only compute a subset of waveforms to speed up the calculation. The `waveform_idxs` property contains the spike indexes associated with the waveforms.

In [ ]:
sorting_MS4.get_shared_unit_spike_feature_names()

In [ ]:
print(sorting_MS4.get_unit_spike_features(1, 'waveforms_idxs'))

Since waveforms are already computed, the next time we (or another function - e.g. `get_unit_templates()`) call it it will just return the stored waveforms.

In [ ]:
waveforms = st.postprocessing.get_unit_waveforms(r_cache, sorting_MS4, verbose=True)

Lighthing fast! 

If we want to recompute the waveforms, for example because we want to extract the waveforms divided by group, we can use the `recompute_info` argument (available for all `postprocessing`, `validation`, and `curation` functions).

Where are waveforms stored? We have seen above that each `Extractor` object has an associated tmp folder. Waveforms (and other features, e.g. pca scores) are stored in this folder as binary files:

In [ ]:
tmp_folder = sorting_MS4.get_tmp_folder()
print(tmp_folder)
print([(p.name) for p in tmp_folder.iterdir()])

In [ ]:
waveforms_group = st.postprocessing.get_unit_waveforms(
    r_cache,
    sorting_MS4,
    max_spikes_per_unit=None, 
    grouping_property='group',
    recompute_info=True,
    verbose=True
)

In [ ]:
sorting_MS4.get_shared_unit_property_names()

In [ ]:
for wf in waveforms_group:
    print(wf.shape)

We can use `spikewidgets` to quickly inspect the spike sorting output:

In [ ]:
unit_ids = sorting_MS4.get_unit_ids()

In [ ]:
w_wf = sw.plot_unit_templates(sorting=sorting_MS4, recording=r_cache, unit_ids=unit_ids[0:6])

In [ ]:
w_acc = sw.plot_autocorrelograms(sorting_MS4, unit_ids=unit_ids[0:6])

### Compute extracellular features
[back to index](#index)

Extracellular features, such as peak to valley duration or full-width half maximum, are important to classify neurons into putative classes (excitatory - inhibitory). The `postprocessing` module of `spiketoolkit` allows one to compute several of these features:

In [ ]:
st.postprocessing.get_template_features_list()

In [ ]:
features = st.postprocessing.compute_unit_template_features(
    r_cache, 
    sorting_MS4, 
    as_dataframe=True, 
    upsampling_factor=10
)
display(features)

For more information about these waveform features, we refer to this [documentation](https://github.com/AllenInstitute/ecephys_spike_sorting/tree/master/ecephys_spike_sorting/modules/mean_waveforms) from the Allen Institute.

### Validation
[back to index](#index)

The `spiketoolkit` package also provides several functions to compute qualitity metrics about the spike sorting results through the `validation` module.

Let's see what metrics are available:

In [ ]:
st.validation.get_quality_metrics_list()

We can compute a subset of quality metrics using the `compute_quality_metrics` function. 


In [ ]:
quality_metrics = st.validation.compute_quality_metrics(
    sorting_MS4, 
    r_cache, 
    metric_names=['firing_rate', 'isi_violation', 'snr'], 
    as_dataframe=True
)
display(quality_metrics)

<a id="spike_curation"></a>
# 7. Curation
[back to index](#index)

### 1) Manual curation using Phy

To perform manual curation we will export the data to [Phy](https://github.com/cortex-lab/phy). 

In [ ]:
st.postprocessing.export_to_phy(
    r_cache, 
    sorting_MS4, 
    output_folder='phy_MS4',
    grouping_property='group', 
    verbose=True, 
    recompute_info=True
)

If you have trouble using the GUI from phy refer to the instructions here: https://github.com/cortex-lab/phy.

In [ ]:
!phy template-gui phy_MS4/params.py

After curating the results we can reload it using the PhySortingExtractor:

In [ ]:
sorting_MS4_phy_curated = se.PhySortingExtractor('phy_MS4/', exclude_cluster_groups=['noise'])

In [ ]:
print(len(sorting_MS4_phy_curated.get_unit_ids()))
print(f"Unit ids after manual curation: {sorting_MS4_phy_curated.get_unit_ids()}")

In [ ]:
_ = [print(sorting_MS4_phy_curated.get_unit_property(i, 'quality')) for i in sorting_MS4_phy_curated.get_unit_ids()]

### b) Automatic curation based on quality metrics
[back to index](#index)

We will demonstrate here how to select subsets of candidate units based on the quality metrics inter-spike interval (isi) and signal to noise ratio (snr).

In [ ]:
st.curation.threshold_isi_violations?

In [ ]:
snr_thresh = 4
isi_viol_thresh = 0.0015
duration = r_cache.get_num_frames()

In [ ]:
isi_violations = st.validation.compute_isi_violations(
    sorting_MS4, 
    duration_in_frames=r_cache.get_num_frames(), 
    isi_threshold=0.0015
)

In [ ]:
isi_violations

In [ ]:
sorting_auto_MS4 = st.curation.threshold_isi_violations(sorting_MS4, isi_viol_thresh, 'greater', duration)
sorting_auto_SC = st.curation.threshold_isi_violations(sorting_SC, isi_viol_thresh, 'greater', duration)

In [ ]:
print('Number of units Mountainsort4: {}'.format(len(sorting_auto_MS4.get_unit_ids())))
print('Number of units Spyking Circus: {}'.format(len(sorting_auto_SC.get_unit_ids())))

In [ ]:
sorting_auto_MS4 = st.curation.threshold_snrs(sorting_auto_MS4, r_cache, snr_thresh, 'less')
sorting_auto_SC = st.curation.threshold_snrs(sorting_auto_SC, r_cache, snr_thresh, 'less')

In [ ]:
print('Number of units Mountainsort4: {}'.format(len(sorting_auto_MS4.get_unit_ids())))
print('Number of units Spyking Circus: {}'.format(len(sorting_auto_SC.get_unit_ids())))

### c) Consensus-based curation
[back to index](#index)

Can we combine the output of multiple sorters to curate the spike sorting output?

Yes, with the `spikecomparison` module we can compare and match the output spike trains of the different sorters, and retrieve a new sorting extractor object containing only the units that were in agreement.

In [ ]:
mcmp = sc.compare_multiple_sorters(
    [sorting_SC, sorting_MS4], 
    ['SC', 'MS4'], 
    spiketrain_mode='union',
    verbose=True
)

In [ ]:
w = sw.plot_multicomp_agreement(mcmp)
w = sw.plot_multicomp_agreement_by_sorter(mcmp)

In [ ]:
agreement_sorting = mcmp.get_agreement_sorting(minimum_agreement_count=2)

In [ ]:
agreement_sorting.get_unit_ids()

In [ ]:
cmp_manual_agr = sc.compare_sorter_to_ground_truth(sorting_MS4_phy_curated, agreement_sorting)

In [ ]:
w_agr = sw.plot_agreement_matrix(cmp_manual_agr)

In [ ]:
w_wf = sw.plot_unit_templates(sorting=agreement_sorting, recording=r_cache, unit_ids=[1])

In [ ]:
st.postprocessing.export_to_phy(
    r_cache, 
    agreement_sorting, 
    output_folder='phy_AGR',
    grouping_property='group', 
    verbose=True, 
    recompute_info=True
)

<a id="export_nwb"></a>
# 7. Save to / load from NWB
[back to index](#index)

In this section we briefly go through the process of exporting recording extractor and sorting extractor objects to the `nwb` file format. We will use the `spikeextractors` classes `NwbRecordingExtractor` and `NwbSortingExtractor`. It can be useful to pass metadata to the `nwb` file during export (i.e. data that describes the ecephys data in this case) and we illustrate this here by passing information about the recording device.

In [ ]:
from pynwb import NWBFile, NWBHDF5IO
from pathlib import Path
from nwb_conversion_tools.utils.spike_interface import write_recording, write_sorting

In [ ]:
metadata = {'Ecephys': {'Device': [{'name': 'Axona',
                                    'description': 'Axona DacqUSB acquisition system'}]}}

In [ ]:
nwb_dir = Path(dir_name, 'nwb')

nwb_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
write_recording(r_cache, nwb_dir / 'axona_data.nwb', metadata=metadata)

In [ ]:
write_sorting(sorting_MS4_phy_curated, nwb_dir / 'axona_data.nwb')

(Re-)load data into recording and sorting extractor.

In [ ]:
recording_nwb = se.NwbRecordingExtractor(nwb_dir / 'axona_data.nwb')

In [ ]:
sorting_nwb = se.NwbSortingExtractor(nwb_dir / 'axona_data.nwb', sampling_frequency=48000)

<a id="export_tint"></a>
# 8. Save to / load from TINT
[back to index](#index)

We can export the spike sorted data back to the 'TINT' format in the form of `.X`, `.cut` and `.clu` files. The `.X` files are similar to the tetrode files generated by the Axona acquisition system, except that the spike times are now based on the spike sorting extractor used for the conversion. The `.cut` and `.clu` files contain the unit labels of each spike timestamp.

For the conversion to 'TINT' both a sorting extractor and recording extractor are required. Spike times are read from the sorting extractor and the corresponding waveforms are extracted from the recording extractor. It is always preferable to use a recording extractor based on `.bin` data, because spike times extracted from spike sorters might not exactly correspond to spike times from the thresholding procedure that created the `.X` files from the raw data. When spike times do not correspond exactly, part (or in the worst caes all) of the waveforms will consist of Gaussian noise if based on an `AxonaUnitRecordingExtractor`, because time periods between waveforms are filled with noise when the extractor object is created. 

The `TintConverter` class has the `compare_timestamps_after_conversion()` method to assess the correspondence of timestamps between the `.X` files created from simple thresholding and the `.X` files created using `TintConverter` after spike sorting.

We recommend creating a new folder for the output from `TintConverter` if you do not want to overwrite existing `.X` files. In this case you also need to copy the `.set` file into that new folder. 

In [ ]:
from hussaini_lab_to_nwb import TintConverter

We load the recording from `unit` data with no noise to later compare how well the waveforms from the thresholding procedure correspond to the waveforms after spike sorting. To be able to do this comparison we save the `TintConverter` output to a subfolder `tint_conversion`. We also need to copy the set file into this folder to be able to read the data back in afterwards.

In [ ]:
if use_raw_data:
    recording = se.AxonaRecordingExtractor(filename=filename)
else:
    recording = se.AxonaUnitRecordingExtractor(filename=filename, noise_std=0)

sorting = se.NwbSortingExtractor(nwb_dir / 'axona_data.nwb', sampling_frequency=48000)

Create new folder for converted data and copy `.set` file into it.

In [ ]:
import shutil

output_dir = Path(dir_name, 'tint_conversion')

output_dir.mkdir(parents=True, exist_ok=True)

shutil.copyfile(filename, output_dir / Path(filename).name)

Convert spike sorted data to TINT format (`.X`, `.cut`, `.clu`). Note that this step can take a while depending on the size of the recording!

In [ ]:
tc = TintConverter()
tc.write_to_tint(recording=recording,
                 sorting=sorting,
                 set_file=output_dir / Path(filename).name)

Finally, we can compare the timestamps from the spike sorted data with the timestamps from the thresholded data (original `.X` files).

In [ ]:
if not use_raw_data:

    df = tc.compare_timestamps_after_conversion(
        filename_old=filename,
        filename_new=output_dir / Path(filename).name
    )

    display(df.T)

---
I hope this notebook was helpful to you. If you have any suggestions, questions or concerns do not hesitate to contact me at `steffen.burgers@catalystneuro.com`.